In [23]:
from datasets import load_dataset
from transformers import AutoTokenizer
import csv
import pandas as pd
import numpy as np

In [9]:
questions_csv = pd.read_csv('./deepseekr1_anatomy_predictions.csv')
questions_csv

,question_idx,question,answer_choices,full_prompt,correct_answer,full_cot,predicted_answer,category
0,0,Which of the following is a disorder character...,"[""Dyslexia"", ""Epilepsy"", ""Hydrocephalus"", ""Nar...","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",3,"Okay, so I've got this question here about a d...",3,anatomy
1,1,Which statement concerning the atrioventricula...,"[""It is formed of autonomic nerve processes an...","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",1,"Okay, so I have this question about the atriov...",3,anatomy
2,2,"A ""dished face"" profile is often associated with","[""a protruding mandible due to reactivation of...","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",1,"Okay, so I have this question about a ""dished ...",0,anatomy
3,3,Gastrulation is the process of,"[""mesoderm formation and occurs after neurulat...","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",1,"Okay, so I need to figure out the correct answ...",0,anatomy
4,4,The palatine processes begin to fuse at,"[""six weeks post-fertilization."", ""eight weeks...","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",2,"Okay, so I have this question about the palati...",2,anatomy
5,5,"In relation to the rib, the corresponding inte...","[""deep to its superior border."", ""superficial ...","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",2,"Okay, so I'm trying to figure out this questio...",1,anatomy
6,6,Which of the following substances is found in ...,"[""Carbon dioxide"", ""Carbon monoxide"", ""Nitroge...","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",2,"Okay, so I have this question here: ""Which of ...",3,anatomy
7,7,Palatine shelf elevation is due to the effect of,"[""descent of the tongue."", ""changes in flexure...","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",3,"Okay, so I'm trying to figure out why Palatine...",2,anatomy
8,8,Which of the following terms describes the mot...,"[""Abduction"", ""Eversion"", ""Flexion"", ""Pronation""]","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",2,"Okay, so I have this question about the motion...",2,anatomy
9,9,Which of the following best describes the stru...,"[""Bladder"", ""Kidney"", ""Ureter"", ""Urethra""]","<｜begin▁of▁sentence｜>You are DeepSeek-R1, an A...",0,"Okay, so I need to figure out which of the opt...",0,anatomy


In [ ]:
# Assuming DeepSeek-R1-0528_mmlu-redux_results.csv exists with the following columns:
# question_idx, question, answer_choices, full_prompt, correct_answer, full_cot, predicted_answer, category

def generate_fake_probing_data(questions_csv):
    early_decoders = [
        "probe_model_answer_correct", "probe_model_answer", "probe_correct_answer",
        "observer_model_answer_correct", "observer_model_answer", "observer_correct_answer",
    ]

    def generate_fake_data(early_decoder):
        if early_decoder.endswith('model_answer_correct'): # did the model answer correctly?
            return np.random.dirichlet(np.ones(2)).tolist()
        elif early_decoder.endswith('model_answer'): # what did the model answer?
            return np.random.dirichlet(np.ones(4)).tolist() 
        elif early_decoder.endswith('correct_answer'): # what was the correct answer?
            return np.random.dirichlet(np.ones(4)).tolist()

    questions_df = pd.read_csv(questions_csv)
    tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-0528")

    data = []
    for row in questions_df.iterrows():
        question_idx = row[1]['question_idx']
        question = row[1]['question']
        answer_choices = row[1]['answer_choices']
        prompt = row[1]['full_prompt']
        correct_answer = row[1]['correct_answer']
        full_cot = row[1]['full_cot']
        predicted_answer = row[1]['predicted_answer']
        category = row[1]['category']

        for early_decoder in early_decoders:
            full_cot_tokens = tokenizer.encode(full_cot)
            for i, token in enumerate(full_cot_tokens):
                token_text = tokenizer.decode(token, skip_special_tokens=True)
                cleaned_token_text = token_text.replace('\n', '\\n').replace('\r', '\\r').replace('\t', '\\t')
                probe_output = generate_fake_data(early_decoder)
                data.append([question_idx, token, cleaned_token_text, early_decoder, probe_output])
    df = pd.DataFrame(data, columns=['question_idx', 'token', 'token_text', 'early_decoder', 'probe_output'])
    df.to_csv('deepseekr1_anatomy_probe_results.csv', index=False, quoting=csv.QUOTE_ALL)

generate_fake_probing_data('deepseekr1_anatomy_predictions.csv')

In [ ]:
for row in questions_df.iterrows():
    print(row[0])
    print(row[1]['question'])
    break

0
0
